In [2]:
# Import modules
import tensorflow as tf
import numpy as np

In [4]:
# Reset graph
tf.reset_default_graph()

In [8]:
# Create a new graph
graph = tf.Graph()

# Set graph as default
with graph.as_default():
    # Create name scope "variables"
    with tf.name_scope("variables"):
        # Variable to keep track how many times 
        # the graph is run
        global_step = tf.Variable(0,
                                 dtype=tf.int32,
                                 trainable=False,
                                  name="global_step"
                                 )
        # Variable to keep track of total output
        total_output = tf.Variable(0,
                                  dtype=tf.float32,
                                   trainable=False,
                                   name="total_output"
                                  )
    # Transformation operations
    with tf.name_scope("transformations"):
        # Separate input layer
        with tf.name_scope("input"):
            # Create input placeholder
            # Takes in a vector
            a = tf.placeholder(
            tf.float32,
                shape = [None],
                name = "input_a"
            )
        # Separate middle layer
        with tf.name_scope("intermediate"):
            b = tf.reduce_prod(a,
                              name = "product_b")
            c = tf.reduce_sum(a,
                             name = "sum_c")
        
        # Separate output layer
        with tf.name_scope("output"):
            output = tf.add(b,c,
                           name = "output")
    
    # Update operations
    with tf.name_scope("update"):
        # Increment total_output variable
        update_total = total_output.assign_add(
        output
        )
        update_step = global_step.assign_add(1)
    
    # Summary operations
    with tf.name_scope("summary"):
        avg = tf.divide(
        update_total, 
            tf.cast(update_step,tf.float32),
            name = "average"
        )
        
        # Create summaries for output node
        tf.summary.scalar('output_summary',output)
        tf.summary.scalar('total_summary',update_total)
        tf.summary.scalar('average_summary',avg)
        
    # Global operations
    with tf.name_scope("global_ops"):
        # Initialization op
        init = tf.global_variables_initializer()
        # Merge all summaries into one operation
        merged_summaries = tf.summary.merge_all()

In [9]:
# Start a Session, using the explicitly created Graph
sess = tf.Session(graph=graph)

# Open a SummaryWriter to save summaries
writer = tf.summary.FileWriter('./improved_graph',graph)

# Initialize Variables
sess.run(init)

In [12]:
def run_graph(input_tensor):
    """
    Helper function; runs the graph with given input tensor and saves summaries
    """
    feed_dict = {a: input_tensor}
    out, step, summary = sess.run([output, update_step, merged_summaries], feed_dict=feed_dict)
    writer.add_summary(summary, global_step=step)

In [13]:
# Run the graph with various inputs
run_graph([2,8])
run_graph([3,1,3,3])
run_graph([8])
run_graph([1,2,3])
run_graph([11,4])
run_graph([4,1])
run_graph([7,3,1])
run_graph([6,3])
run_graph([0,2])
run_graph([4,5,6])

In [14]:
# Write the summaries to disk
writer.flush()


In [15]:
# Close the SummaryWriter
writer.close()

In [16]:
# Close the session
sess.close()

To start TensorBoard after running this code, run the following command:

`tensorboard --logdir='./improved_graph'`